# CNN Architectures

## 1. VGG (Visual Geometry Group)

Key points:
-  With Each Conv Layer increase the number of Filters used ie if used 16filters in 1st layer,use 16 or more filters in second layer
-  In VGG mostly 3* 3 are used , as it is cheaper to use then 5 * 5 filters and covers most of area


![vgg](vgg_cropped.png "VGG")

### Creating Custom VGG in Keras

In [ ]:
import tensorflow 
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Flatten, Activation,Input,BatchNormalization,add

In [ ]:
custom_vgg=Sequential(name='VGG')
custom_vgg.add(Conv2D(64,(3,3),strides=1,padding='same',activation='relu',input_shape=(32,32,3)))
custom_vgg.add(Dropout(0.4))
custom_vgg.add(Conv2D(64,(3,3),strides=1,padding='same',activation='relu'))
custom_vgg.add(Dropout(0.4))
custom_vgg.add(MaxPooling2D((2,2)))

custom_vgg.add(Conv2D(128,(3,3),strides=1,padding='same',activation='relu'))
custom_vgg.add(Dropout(0.4))
custom_vgg.add(Conv2D(128,(3,3),strides=1,padding='same',activation='relu'))
custom_vgg.add(Dropout(0.4))
custom_vgg.add(MaxPooling2D((2,2)))

custom_vgg.add(Conv2D(256,(3,3),strides=1,padding='same',activation='relu'))
custom_vgg.add(Dropout(0.4))
custom_vgg.add(Conv2D(256,(3,3),strides=1,padding='same',activation='relu'))
custom_vgg.add(Dropout(0.4))
custom_vgg.add(MaxPooling2D((2,2)))

custom_vgg.add(Flatten())
custom_vgg.add(Dense(10,activation='softmax'))

custom_vgg.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

custom_vgg.summary()



Model: "VGG"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
dropout_7 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
dropout_8 (Dropout)          (None, 16, 16, 128)       0       

-  **Cannot Create Deeper Architectures using vgg due "Vanishing gradient Problem"**
-  Useful for Transfer Learning and Small Classification tasks


***

## 2. ResNet ( Residual Network )

Key Points :
 - First Deep Layer Network with 152 Layers
 - **Connects current layer with previous layer as well as layer behind previous layer**
 
In VGG, every layer is connected to its previous layer, from which it is getting its inputs. This makes sure that upon propagation from layer to layer, more and more useful features are carried and less important features are dropped out. This is not the best way since the latter layers cannot see what the former layers have seen

![](resnet.png "Resnet")
<br>
<br>
**The core idea is, let us consider x as an output of some Conv2D layer.**
**Add few Conv2D layers to x and then add the output to x and send this as input to the next layer**

![](resnet2.png "resnet")
<br>
#### x = x + conv2d ===> next layer
<br>

**Training of such a deep Residual Networks is possible by using BatchNormalisation layers after every Convolutional Layer. Batch Normalisation layers will boost the values of weights and hence higher learning rates can be used while training, which will help train faster and also can minimize Vanishing Gradient problem**

### Creating Custom Resnet Network

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,BatchNormalization,add

In [ ]:
stride=1
channel_axis=3

def res_layer(x,filters,pooling=False,dropout=0.0):
    temp=x
    temp=Conv2D(filters,(3,3),strides=stride,padding='same')(temp)
    temp=BatchNormalization(axis=channel_axis)(temp)
    temp=Activation('relu')(temp)
    temp=Conv2D(filters,(3,3),strides=stride,padding='same')(temp)
    
    x=add([temp,Conv2D(filters,(3,3),strides=stride,padding='same')(x)])
    
    if pooling:
        x=MaxPooling2D((2,2))(x)
    if dropout!=0.0:
        x=Dropout(dropout)(x)
    x=BatchNormalization(axis=channel_axis)(x)
    x=Activation('relu')(x)
    return x

And the process is, it connects the input layer to a Conv2D layer having filters specified in our function calling and then attaches a BatchNormalization layer, upon which a ReLU Activation layer is added, and then another Conv2D layer is stacked. Now, this stacked output is added with the initial input, but the initial input is transformed by passing it through a Conv2D layer of given filters. This step is done to match the output sizes of both the layers that are going to be added. Then, if we prefer having a MaxPooling2D layer, we add that and also if any Dropout value is given, then a Dropout layer is also added and then finally one more BatchNormalization layer and Activation layer are added and then this final layer is returned by our res_layer function.


![](resnet3.png 'resnet')

In [ ]:
inp=Input(shape=(32,32,3))
x=inp
x=Conv2D(16,(3,3),strides=stride,padding='same')(x)
x=BatchNormalization(axis=channel_axis)(x)
x=Activation('relu')(x)

x=res_layer(x,32,dropout=0.2)
x=res_layer(x,32,dropout=0.3)
x=res_layer(x,32,dropout=0.4,pooling=True)
x=res_layer(x,64,dropout=0.2)
x=res_layer(x,64,dropout=0.2,pooling=True)
x=res_layer(x,256,dropout=0.4)
x=Flatten()(x)
x=Dropout(0.4)(x)
x=Dense(4096,activation='relu')(x)
x=Dropout(0.23)(x)
x=Dense(100,activation='softmax')(x)

resnet_model=Model(inp,x,name='Resnet')
resnet_model.summary()


Model: "Resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_42 (Conv2D)              (None, 32, 32, 16)   448         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_19 (BatchNo (None, 32, 32, 16)   64          conv2d_42[0][0]                  
__________________________________________________________________________________________________
activation_19 (Activation)      (None, 32, 32, 16)   0           batch_normalization_19[0][0]     
_____________________________________________________________________________________________

## 3. Dense Net

In ResNet, we added the stacked layer along with its input layer. <br>**In DenseNet, for a given layer, all other layers preceding to it are concatenated and given as input to the current layer.**<br> With such an arrangement, we can use smaller filter counts and also, this will minimize the vanishing gradient problem as all layers are directly connected to the output, gradients can be calculated directly from the output for each layer.


In [ ]:
import tensorflow
from keras.layers import concatenate,Input,BatchNormalization,Dense,Conv2D,Flatten,add,Activation,MaxPooling2D,Dropout
from keras.models import Model

### Creating Custom Dense Net

In [ ]:
def dense_layer(x, layer_configs):
    layers=[]

    for i in range(2):
        if layer_configs[i]["layer_type"] == "Conv2D":
            layer = Conv2D(layer_configs[i]["filters"], layer_configs[i]["kernel_size"], strides = layer_configs[i]["strides"], padding = layer_configs[i]["padding"], activation = layer_configs[i]["activation"])(x)
        layers.append(layer)

    for n in range(2, len(layer_configs)):
        if layer_configs[n]["layer_type"] == "Conv2D":
            layer = Conv2D(layer_configs[n]["filters"], layer_configs[n]["kernel_size"], strides = layer_configs[n]["strides"], padding = layer_configs[n]["padding"], activation = layer_configs[n]["activation"])(concatenate(layers, axis = 3))
        layers.append(layer)

    return layer

In [ ]:

layer_f8 = [{"layer_type" : "Conv2D", "filters" : 8, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 8, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 8, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 8, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 8, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"}]


layer_f16 =  [{"layer_type" : "Conv2D", "filters" : 16, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 16, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 16, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 16, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 16, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"}]


layer_f32 =  [{"layer_type" : "Conv2D", "filters" : 32, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 32, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 32, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 32, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 32, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"}]



layer_f64 = [{"layer_type" : "Conv2D", "filters" : 64, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 64, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 64, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 64, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 64, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"}]



layer_f128 =  [{"layer_type" : "Conv2D", "filters" : 128, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 128, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 128, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 128, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"},
            {"layer_type" : "Conv2D", "filters" : 128, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"}]


In [ ]:
inp=Input((32,32,3))
x=inp
x = Conv2D(4, (3, 3), strides=1, padding="same", activation ="relu")(x)
x = dense_layer(x, layer_f8)
x = Dropout(0.8)(x)

x = BatchNormalization(axis = 3)(x)
x = dense_layer(x, layer_f16)
x = Dropout(0.8)(x)

x = BatchNormalization(axis = 3)(x)
x = dense_layer(x, layer_f32)
x = Dropout(0.8)(x)

x = BatchNormalization(axis = 3)(x)
x = dense_layer(x, layer_f64)
x = Dropout(0.8)(x)



x = BatchNormalization(axis = 3)(x)
x = dense_layer(x, layer_f128)
x = Dropout(0.8)(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization(axis = 3)(x)
x = Conv2D(96, (1, 1), activation = "relu")(x)
x = BatchNormalization(axis = 3)(x)



x = MaxPooling2D((2, 2))(x)
x = BatchNormalization(axis = 3)(x)
x = Flatten()(x)



x = Dropout(0.4)(x)
x = Dense(14, activation = "softmax")(x)



dense_net = Model(inp, x,name='DenseNet')

dense_net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_55 (Conv2D)              (None, 32, 32, 4)    112         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_56 (Conv2D)              (None, 32, 32, 8)    296         conv2d_55[0][0]                  
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 32, 32, 8)    296         conv2d_55[0][0]                  
__________________________________________________________________________________________________
concatenat

## 4. Inception Net
- In ResNet, we created deeper networks. The idea of Inception Net is to make the network wider. This can be done by parallel connection of multiple layers having different filters, and then finally concatenating all of those parallel paths to pass to next layers.

In [ ]:
from keras.layers import AveragePooling2D
def inception_layer(x, layer_configs):
  layers = []

  for configs in layer_configs:
    if configs[0]["layer_type"] == "Conv2D":
      layer = Conv2D(configs[0]["filters"], configs[0]["kernel_size"], strides = configs[0]["strides"], padding = configs[0]["padding"], activation = configs[0]["activation"])(x)

    if configs[0]["layer_type"] == "MaxPooling2D":
      layer = MaxPooling2D(configs[0]["kernel_size"], strides = configs[0]["strides"], padding = configs[0]["padding"])(x)

    for n in range(1, len(configs)):
      if configs[n]["layer_type"] == "Conv2D":
        layer = Conv2D(configs[n]["filters"], configs[n]["kernel_size"], strides = configs[n]["strides"], padding = configs[n]["padding"], activation = configs[n]["activation"])(layer)

      if configs[n]["layer_type"] == "MaxPooling2D":
        layer = MaxPooling2D(configs[n]["kernel_size"], strides = configs[n]["strides"], padding = configs[n]["padding"])(layer)

    layers.append(layer)

  return concatenate(layers, axis = 3)

In [ ]:
layer_3a = [[{"layer_type" : "Conv2D", "filters" : 64, "kernel_size" : (1, 1), "strides" : 1, "padding" : "same", "activation" : "relu"}],
[{"layer_type" : "Conv2D", "filters" : 96, "kernel_size" : (1, 1), "strides" : 1, "padding" : "same", "activation" : "relu"},
{"layer_type" : "Conv2D", "filters" : 128, "kernel_size" : (3, 3), "strides" : 1, "padding" : "same", "activation" : "relu"}],
[{"layer_type" : "Conv2D", "filters" : 16, "kernel_size" : (1, 1), "strides" : 1, "padding" : "same", "activation" : "relu"},
{"layer_type" : "Conv2D", "filters" : 32, "kernel_size" : (5, 5), "strides" : 1, "padding" : "same", "activation" : "relu"}],
[{"layer_type" : "MaxPooling2D","strides" : 1,"kernel_size" : (3, 3),"padding" : "same"},
{"layer_type" : "Conv2D","filters" : 32,"kernel_size" : (1, 1),"strides" : 1,"padding" : "same","activation" : "relu"}]]



layer_3b = [[{"layer_type" : "Conv2D","filters" : 128,"kernel_size" : (1, 1),"strides" : 1,"padding" : "same","activation" : "relu" }],
[{ "layer_type" : "Conv2D","filters" : 128,"kernel_size" : (1, 1),"strides" : 1,"padding" : "same","activation" : "relu" },
{"layer_type" : "Conv2D","filters" : 192,"kernel_size" : (3, 3),"strides" : 1,"padding" : "same","activation" : "relu"}],
[{"layer_type" : "Conv2D","filters" : 32,"kernel_size" : (1, 1),"strides" : 1,"padding" : "same","activation" : "relu"},
{"layer_type" : "Conv2D","filters" : 96,"kernel_size" : (5, 5),"strides" : 1,"padding" : "same","activation" : "relu" }],
[{"layer_type" : "MaxPooling2D","strides" : 1,"kernel_size" : (3, 3),"padding" : "same"},
{"layer_type" : "Conv2D","filters" : 96,"kernel_size" : (1, 1),"strides" : 1,"padding" : "same","activation" : "relu"}]]



layer_4a = [[{"layer_type" : "Conv2D","filters" : 192,"kernel_size" : (1, 1),"strides" : 1,"padding" : "same","activation" : "relu"}],
[{"layer_type" : "Conv2D","filters" : 96,"kernel_size" : (1, 1),"strides" : 1,"padding" : "same","activation" : "relu"},
{"layer_type" : "Conv2D","filters" : 208,"kernel_size" : (3, 3),"strides" : 1,"padding" : "same","activation" : "relu"}],
[{"layer_type" : "Conv2D","filters" : 16,"kernel_size" : (1, 1),"strides" : 1,"padding" : "same","activation" : "relu"},
{"layer_type" : "Conv2D","filters" : 48,"kernel_size" : (5, 5),"strides" : 1,"padding" : "same","activation" : "relu"}],
[{"layer_type" : "MaxPooling2D","strides" : 1,"kernel_size" : (3, 3),"padding" : "same"},
{"layer_type" : "Conv2D","filters" : 64,"kernel_size" : (1, 1),"strides" : 1,"padding" : "same","activation" : "relu"}]]



layer_4b = [[{"layer_type" : "Conv2D","filters" : 160,"kernel_size" : (1, 1),"strides" : 1,"padding" : "same","activation" : "relu"}],
[{"layer_type" : "Conv2D","filters" : 112,"kernel_size" : (1, 1),"strides" : 1,"padding" : "same","activation" : "relu"},
{"layer_type" : "Conv2D","filters" : 224,"kernel_size" : (3, 3),"strides" : 1,"padding" : "same","activation" : "relu"}],
[{"layer_type" : "Conv2D","filters" : 24,"kernel_size" : (1, 1),"strides" : 1,"padding" : "same","activation" : "relu"},
{"layer_type" : "Conv2D","filters" : 64,"kernel_size" : (5, 5),"strides" : 1,"padding" : "same","activation" : "relu"}],
[{"layer_type" : "MaxPooling2D","strides" : 1,"kernel_size" : (3, 3),"padding" : "same"},
{"layer_type" : "Conv2D","filters" : 64,"kernel_size" : (1, 1),"strides" : 1,"padding" : "same","activation" : "relu"}]]

In [ ]:

inp = Input(shape = (32, 32, 3))

x = inp

x = Conv2D(64, (7, 7), strides = 2, padding = "same", activation = "relu")(x)

x = MaxPooling2D((3, 3), padding = "same", strides = 2)(x)

x = Conv2D(64, (1, 1), strides = 1, padding = "same", activation = "relu")(x)

x = Conv2D(192, (3, 3), strides = 1, padding = "same", activation = "relu")(x)

x = MaxPooling2D((3, 3), padding = "same", strides = 2)(x)

x = inception_layer(x, layer_3a)

x = inception_layer(x, layer_3b)

x = MaxPooling2D((3, 3), padding = "same", strides = 2)(x)

x = inception_layer(x, layer_4a)



x1 = AveragePooling2D((2, 2), strides = 3)(x)

x1 = Conv2D(128, (1, 1), padding = "same", activation = "relu")(x1)

x1 = Flatten()(x1)

x1 = Dense(1024, activation = "relu")(x1)

x1 = Dropout(0.7)(x1)

x1 = Dense(100, activation = "softmax")(x1)



inc = Model(inp, x1)

inc.summary()

W0828 03:04:36.163210 139641907955584 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_103 (Conv2D)             (None, 16, 16, 64)   9472        input_5[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_13 (MaxPooling2D) (None, 8, 8, 64)     0           conv2d_103[0][0]                 
__________________________________________________________________________________________________
conv2d_104 (Conv2D)             (None, 8, 8, 64)     4160        max_pooling2d_13[0][0]           
__________________________________________________________________________________________________
conv2d_105

There are many variants of Inception Nets. The differences among them are:<br>
- Instead of using a 5*5 filter, use two 3*3 filters as they are computationally efficient (as discussed in the VGGNet).
- Using a 1 * 1 Conv2D layer with smaller filter count before performing any Conv2D layer with larger filter sizes as 1 * 1 filter with less filter count will reduce the depth of the input and hence is computationally efficient.
-  Instead of performing a 3 *  3 filter, perform 1 * 3 filter followed by a 3 * 1 filter. This will drastically improve the computational efficiency.

**InceptionNets are preferable as they are not just deeper, but also wider and we can stack many such layers and still the output params to be trained are less when compared to all other architectures.**

## 5. Xception Net
- Xception Net is an improvisation of InceptionNet in terms of computational efficiency. Xception means Extreme Inception

- Xception Net outperforms Inception Net v3

- In Inception Net normal convolutional operations are performed whereas in Xception Net, Depthwise Separable Convolutional operations are performed.
- In Depthwise separable convolutions, each channel have only one kernel to do convolution. Hence, by performing Depthwise Separable Convolutions, we can reduce the **computational complexity** as every kernel is of two dimensional only and is convoluting only over one channel

In [ ]:
from keras.layers import DepthwiseConv2D
inp = Input(shape = (32, 32, 3))

x = inp

x = Conv2D(32, (3, 3), strides = 2, padding = "same", activation = "relu")(x)

x = BatchNormalization(axis = 3)(x)

x = Dropout(0.4)(x)
x = Conv2D(64, (3, 3), strides = 1, padding = "same", activation = "relu")(x)

x = BatchNormalization(axis = 3)(x)

x = Dropout(0.4)(x)



x1 = DepthwiseConv2D((3, 3), (1, 1), padding = "same", activation = "relu")(x)

x = BatchNormalization(axis = 3)(x)

x = Dropout(0.4)(x)

x1 = DepthwiseConv2D((3, 3), (1, 1), padding = "same", activation = "relu")(x1)

x = BatchNormalization(axis = 3)(x)

x = Dropout(0.4)(x)

x1 = MaxPooling2D((2, 2), strides = 1)(x1)



x = concatenate([x1, Conv2D(64, (2, 2), strides = 1)(x)])



x1 = Activation("relu")(x)

x1 = Conv2D(256, (3, 3), strides = 1, padding = "same", activation = "relu")(x1)

x = BatchNormalization(axis = 3)(x)

x = Dropout(0.4)(x)

x1 = DepthwiseConv2D((3, 3), strides = 1, padding = "same", activation = "relu")(x1)

x = BatchNormalization(axis = 3)(x)

x = Dropout(0.4)(x)

x1 = DepthwiseConv2D((3, 3), strides = 1, padding = "same")(x1)

x = BatchNormalization(axis = 3)(x)

x = Dropout(0.4)(x)

x1 = MaxPooling2D((2, 2), strides = 1)(x1)



x = concatenate([x1, Conv2D(256, (2, 2), strides = 1)(x)])





x = Activation("relu")(x)

x = Conv2D(256, (3, 3), strides = 1, padding = "same", activation = "relu")(x)

x = BatchNormalization(axis = 3)(x)

x = Dropout(0.4)(x)

x = Conv2D(128, (3, 3), strides = 1, padding = "same", activation = "relu")(x)

x = BatchNormalization(axis = 3)(x)

x = Dropout(0.4)(x)

x = Flatten()(x)



x = Dense(100, activation = "softmax")(x)





xception = Model(inp, x)

xception.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_125 (Conv2D)             (None, 16, 16, 32)   896         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_22 (BatchNo (None, 16, 16, 32)   128         conv2d_125[0][0]                 
__________________________________________________________________________________________________
dropout_20 (Dropout)            (None, 16, 16, 32)   0           batch_normalization_22[0][0]     
__________________________________________________________________________________________________
conv2d_126

**We can use XceptionNet in low power devices as there are less compuations in the Conv layer and also is accuracy is pretty similar when compared to a normal convolution layer**